In [1]:
# First we import the libraries
import numpy as np
import matplotlib.pyplot as plt

from qucumber.nn_states import PositiveWaveFunction
from qucumber.callbacks import MetricEvaluator

import qucumber.utils.training_statistics as ts
import qucumber.utils.data as data
import qucumber

import torch

# set random seed on cpu but not gpu, as gpu is not used
qucumber.set_random_seed(1234, cpu=True, gpu=False)

In [39]:
# Now we get the training data
# Training data is given in the data/nY=8/ directory for each delta
# in the '...samples.csv' files.
# So the delta needs to be specified below

#-------------------- Specify delta and nY ---------------------#
delta = "1.04" 
nY = "20" # nY refers to the number of atoms in the array
# For the nY = 8 system case:
# delta is any of 1.00, 1.02, 1.04, 1.06, 1.08, 1.10, 1.12, 1.14, 1.16, 1.18, 1.20
# For nY > 8:
# delta is any of 1.00, 1.04, 1.08, 1.12, 1.16, 1.20, 1.28
#---------------------------------------------------------------#

train_path = "data/nY="+nY+"/δ="+delta+"_samples.csv"
train_data = data.load_data(train_path)

# The training data is stored in train_data[0]
# and the dimension of the data is:
train_data[0].shape

torch.Size([10000, 20])

In [40]:
# So now we have the training data. Our goal is to train a
# Restricted Boltzman Machine using this training data. We use
# QuCumber to create an instance of an RBM. First we need to specify the 
# number of visible nodes (nv) and number of higgen nodes (nh). 
# As we have an array of 8 atoms, we have 6 inputs and so nv = 8
nv = train_data[0].shape[-1]

# Number of hidden nodes of an RBM is a hyperparameter which depends on the 
# data we are using and which can be varied to get optimal result. 
# For this problem, nh was varied from 4 to 8 and nh = 8 gave quite good results.
# Hence, we set nh = 8.
nh = 20

# Finally we create an RBM with nv visible nodes and nh  hidden nodes.
nn_state = PositiveWaveFunction(num_visible=nv, num_hidden=nh, gpu=False)

In [41]:
# Below we have more hyperparameters of the RBM model. Like the number of 
# hidden nodes, the parameters below can be varied to get optimal results.
# The following hyperparameters seemed to work quite well. Note that when
# you have different data, then different values of the hyperparameter will give
# better solutions. 
# Further description of the parameters (if you are curious) is in:
# https://qucumber.readthedocs.io/en/stable/quantum_states.html?highlight=fit#qucumber.nn_states.PositiveWaveFunction.fit

epochs = 500
pbs = 100
nbs = pbs
lr = 0.0065 
k = 10

# Now we train our RBM using the above parameters. 
# Note that the training process will take 2 - 3 minutes

nn_state.fit(
    train_data[0],
    epochs=epochs,
    pos_batch_size=pbs,
    neg_batch_size=nbs,
    lr=lr,
    k=k,
    time=True,
)

Total time elapsed during training: 177.000 s


In [42]:
# After training is complete, we save the parameters of the trained RBM below
# as rydberg_data.pt in the output directory.
nn_state.save("output/rydberg_data.pt")
torch.load("output/rydberg_data.pt")
# Below we have the parameters  (weights and biases) of the trained RBM:

{'rbm_am': OrderedDict([('weights',
               tensor([[-7.2569e-01,  7.8263e-01, -3.6737e-01, -2.4308e-01,  1.2966e-01,
                         1.9648e-01, -9.4116e-01,  8.9861e-01, -6.9678e-01,  3.8171e-01,
                        -3.3002e-01,  3.0464e-01, -2.4221e-01, -2.2639e-02,  4.9681e-01,
                        -1.3452e+00,  1.5575e+00, -1.5080e+00,  9.7990e-01, -1.0042e-01],
                       [-5.8857e-01,  3.9073e-01, -4.1329e-01,  1.6366e-01, -9.8710e-02,
                        -3.2510e-01,  1.7210e-01,  1.7584e-01, -7.8337e-01,  6.2921e-01,
                        -4.0807e-01,  1.9091e-01, -6.8900e-01,  8.1189e-01, -1.0325e+00,
                         8.7235e-01, -7.9897e-01,  4.0410e-01, -5.9696e-01,  5.6091e-01],
                       [ 1.3759e-02, -2.8291e-01,  2.2634e-01, -1.5544e-02, -3.5159e-01,
                         4.3210e-01, -2.7945e-01, -3.7633e-01,  9.3043e-01, -1.6007e+00,
                         1.4509e+00, -1.2232e+00,  7.3229e-01, -7.7170e-

In [43]:
# Now we have our trained RBM. Here we reconstruct the wavefunction by sampling from the RBM. 
# Let us see step by step how it works -

# The first step is to sample from our trained RBM using QuCumber.
# We then get num_samples samples which is stored in the variable 'samples'
num_samples = 5000
samples = nn_state.sample(num_samples = num_samples , k = 10)

# Now we print out the samples in 'output/reconstructedSample.txt'
sampleList = samples.tolist() # convert to list for convenience
sampleList
with open('output/reconstructedSample.txt', 'w') as fp:
    for item in sampleList:
        fp.write("%s\n" % item)

In [44]:
## -------------------------------------------------------------------- ##
## This block of code computes amplitudes. For nY > 8, it is not necessary
## to run this block as computation takes a lot of time.
## -------------------------------------------------------------------- ##
# From the samples we obtain the amplitudes of each of the basis states.
# Note that here we have an 8 atom array. Hencem there are 2^8 basis states
# which are: |00000000>, |00000001>, ..., |11111111>

# Now say X is any of the states from {|00000000>, |00000001>, ..., |11111111>}
# Now, the amplitude of a state X is computed by taking the
# square root of the number of states X present in 
# our produced sample in in 'output/reconstructedSample.txt'.

# Therefore the reconstructed sample is like: 
#    |psi> = amplitudeList[0] |00000000> + amplitudeList[1] |00000001> + ... + amplitudeList[255] |11111111>  
# where amplitudeList is defined below.

# The following functions below finds the amplitudes in the way described above, stores the 
# amplitudes in amplitudeList and then also 
# prints out the amplitudes in 'output/reconstructedSample.txt'

def amplitude(sample):
    return np.sqrt(sampleList.count(sample)/num_samples)

def getBinaryString(i):
    sites = nv # nv = 8
    getbinary = lambda x, n: format(x, 'b').zfill(n)
    tempStr = getbinary(i, sites)
    return tempStr

# Python code to convert string to list character-wise
def ConvertToList(string):
    list1=[]
    list1[:0]=string
    return list1

amplitudeList = []
def getAmplitude(sample):
    for i in range (2**nv):
        binaryString = getBinaryString(i)
        strList = ConvertToList(binaryString)
        tempStr = str(amplitude(list(map(int, strList))))
        amplitudeList.append(tempStr)
    return 0

getAmplitude(sampleList)
with open('output/reconstructedStateAmplitudes.txt', 'w') as fp:
    for item in amplitudeList:
        fp.write("%s\n" % item)

KeyboardInterrupt: 

In [45]:
# So now we have obtained the reconstructed wavefunction.
# Now in the 'data' directory for each delta we have 1_pt_fun and 2_pt_fn
# 1_pt_fn is the one point function and 
# 2_pt_fn is the 2 point function.

# 1_pt_fn is the average occupation of each site. Note that
# there are 8 sites as there are 8 atoms. Occupation of each site
# refers to the proportion of atoms in the up state (state 1) 
# in each state.

# Now we get average occupation of each site by finding the 
# number of atoms in the up state in each site and then
# by dividing by the number of the produced samples 
occupation = [0]*nv
for i in range(len(sampleList)):
    j = 0
    for j in range(nv):
        occupation[j] = sampleList[i][j] + occupation[j]
for i in range(nv):
    occupation[i] = occupation[i]/len(sampleList)

# Thus the average occupation per site of each site for the reconstructed state is given below
# This data can be compared to data/nY=8/δ=delta_1_pt_fn.csv for the relevant delta

# (Note: The list in data/nY=8/δ=delta_1_pt_fn.csv was obtained from 
# the sample in data/nY=8/δ=delta_samples.csv)

# Thus the data in 'occupation' and 'data/nY=8/δ=delta_1_pt_fn.csv' are expected to be close (which they are)
occupation

[0.3774,
 0.3932,
 0.3704,
 0.388,
 0.3596,
 0.4084,
 0.3734,
 0.3836,
 0.3802,
 0.3966,
 0.3816,
 0.3852,
 0.374,
 0.3928,
 0.3564,
 0.3902,
 0.3612,
 0.407,
 0.3654,
 0.3914]

In [32]:
# Now we look at the 2 point function
# 2 point function is the covariance matrix for occupations.
# It is found by 2_pt_fn = outerProdT1 - OuterProdT2

# outerProdT1 is the average of the outer product of the spin vectors with themselves
# So we first find the sum of the outerproducts of the spin vectors with themselves.
# Then we take the average
outerProdT1 = np.zeros((nv,nv))
for i in range(len(sampleList)):
    outerProdT1 = outerProdT1 + np.outer(sampleList[i], sampleList[i])
outerProdT1 = outerProdT1/len(sampleList)

# outerProd2 is simply the outer product of the occupation vector
# with itself. Note that we already found the occupation vector for the 
# one point function

outerProdT2 = np.outer(occupation,occupation)
outerProdT2

#Hence we have:
TwoPointFunc = outerProdT1 - outerProdT2

# The TwoPointFunction data for the reconstructed state is given below.
# This matrix can be compared to data/nY=8/δ=delta_2_pt_fn.csv for the relevant delta

# (Note: The matrix in data/nY=8/δ=delta_2_pt_fn.csv was obtained from 
# the sample in data/nY=8/δ=delta_samples.csv)

# Thus the matrix in 'TwoFuncPoint' and 'data/nY=8/δ=delta_2_pt_fn.csv' are expected 
# to be nearly equal (which they are)

# Note that the matrix in data/nY=8/δ=delta_2_pt_fn.csv may be a bit misleading. 
# The matrix generated there is symmetric and this is why the values 
# below the diagonal were not computed. However the actual matrix is symmetric
TwoPointFunc

array([[ 0.24015936, -0.1357304 ,  0.12719264, -0.11160576,  0.09272128,
        -0.0903248 ,  0.09215216, -0.08236848,  0.09036016, -0.08164624,
         0.08345312, -0.08640496,  0.0898792 , -0.1037352 ,  0.11832848,
        -0.1359312 ],
       [-0.1357304 ,  0.242431  , -0.1351996 ,  0.1282264 , -0.1039392 ,
         0.094522  , -0.0908474 ,  0.0838222 , -0.0865174 ,  0.0857786 ,
        -0.0834268 ,  0.0848394 , -0.087213  ,  0.094953  , -0.1024222 ,
         0.129018  ],
       [ 0.12719264, -0.1351996 ,  0.24175536, -0.13462624,  0.11457472,
        -0.1007352 ,  0.09650984, -0.08521752,  0.08680184, -0.08027176,
         0.08283088, -0.08661704,  0.0811108 , -0.0882548 ,  0.08845752,
        -0.1050088 ],
       [-0.11160576,  0.1282264 , -0.13462624,  0.24138816, -0.13142848,
         0.1236768 , -0.10847056,  0.09100368, -0.08839856,  0.08274384,
        -0.08320192,  0.08939536, -0.0836872 ,  0.0903832 , -0.08576368,
         0.0972192 ],
       [ 0.09272128, -0.1039392 ,  0